In [1]:
# GPU Testing and print related info
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')
else:
    print(device)

NVIDIA GeForce RTX 4080 Laptop GPU
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [2]:
# Upload dataset csv file to python:
import numpy as np
import pandas as pd
file_path = "./Database/New/stocks/A.csv"
stock = pd.read_csv(file_path)
stock['Index'] = stock.reset_index().index
for column in stock:
    print(column)
print(stock.shape)
print(stock)


FileNotFoundError: [Errno 2] No such file or directory: './Database/New/stocks/A.csv'

In [13]:
# Separate the original stock dataset into a number of training blocks
n = len(stock)
rows_in_x = 120
rows_in_y = 10
remainder = n % 120
train_x_stock_set = []
train_y_stock_set = []

start_idx_x = 0
start_idx_y = 0

In [ ]:
for i in range(n/rows_in_x):
    end_idx = start_idx + rows_in_x + rows_in_x
    x_stock_bloc = stock.iloc[start_idx : end_idx]
    y_stock_bloc = stock.iloc[end_idx : end_idx + 10]
    train_x__set = train_x_stock_set.append(x_stock_bloc)
    train_y__set = train_y_stock_set.append(y_stock_bloc)
    

In [16]:
# Select specific columns from a single stock:
C = stock_set[1]["High"]
t = stock_set[1]["Index"]
print(stock)


Empty DataFrame
Columns: [Date, Open, High, Low, Close, Adj Close, Volume, Index]
Index: []


In [17]:
# Algorithm 1:
import math as m
p = [None]*79
u = [None]*79
d = [None]*79
C_pred = [None]*79
for k in t:
    C_pred[k+1] = C[k]*u[k]*p[k] + C[k]*d[k]*(1-p[k])
    # Absolute value means L1-Distance, but also could use L2-Distance.
    diff = m.abs(C_pred[k+1] - C[k+1])


In [18]:
# Algorithm 2:
p = [None]*79
n = [None]*79
C_sub = [None]*79
C_pred = [None]*79
for i in t:
    for j in range(n[i]):
        C_sub[i+j+1] = (C_sub[i+j] + 1*(1/(m.sqrt(n))))*p[i]
        + (C_sub[i+j] + (-1)*(1/(m.sqrt(n))))*(1-p[i])
    C_pred[i+1] = C_sub[i+j+1]
    # L1-Distance
    diff = m.abs(C_pred[i+1] - C[i+1])

In [ ]:
# RNN Training:
import tensorflow as tf

# ... 